## ENSEMBLE LEARNING AND RANDOM FOREST


**1. If you have trained five different models on the exact same training data, and
they all achieve 95% precision, is there any chance that you can combine these
models to get better results? If so, how? If not, why?**

Si, es probable que podamos obtener mejores resultados combinando estos modelos utilizando un Voting Classifier. Aunque los modelos independientes tengan una alta precisión, pueden cometer errores diferentes en instancias distintas. Si lo errores de los modelos no están correlacionados entre sí (independientes), la votación permite que los errores individuales se compensen entre sí. 

**2. What is the difference between hard and soft voting classifiers?**

La diferencia radica en cómo el ensamble decide la clase final. Mientras que el Hard Voting cuenta los votos de cada modelo individuales y elige la clase que recibió la mayoría de votos, el Soft Voting calcula el promedio de las probabilidades estimadas para cada clase por todos los modelos individuales para luego elegir la clase con la probabilidad promedio más alta. Esto implica que, para usarlo, todos los clasificadores individuales deben ser capaces de estimar probabilidades. 

**3. Is it possible to speed up training of a bagging ensemble by distributing it across
multiple servers? What about pasting ensembles, boosting ensembles, random
forests, or stacking ensembles?**

La posibilidad de distribuir el entrenamiento depende de si el algoritmo es independiente o secuencial: 

- Bagging, Pasting y Random Forest: si es posible ya que cada predictor de estos modelos se entrena de forma independiente a los demás utilizando diferentes subconjuntos de datos.
- Boosting: no se puede paralelizar fácilmente (o solo de forma limitada) ya que funcionan de forma secuencial, cada predictor se entrena para corregir los errores cometidos por su predecesor, por loq ue no pueden entrenarse al mimsmo tiempo en servidores distintos.
- Stacking ensembles: su entrenamiento es mayormente secuencial entre capas. Aunque los modelos dentro de una misma capa si podrían entrenarse en paralelo, el proceso global de pasar de una capa a otra es jerárquico y dependiente (los modelos de la capa 2 necesitan las predicciones de los modelos de la capa 1).

**4. What is the benefit of out-of-bag evaluation?**

El benificio principal es que permite evaluar el desempeño de un modelo de bagging sin necesidad de reservar un conjunto de validación por separado. Esto se debe a que, matemáticamente, cada predictori utiliza solo de media el 63% de las instancias de entrenamiento, por lo que el restante son instancias “limpias” que pueden usarse como conjunto de validación, muy útil si la cantidad de datos es escasa.

**5. What makes Extra-Trees more random than regular Random Forests? How can
this extra randomness help? Are Extra-Trees slower or faster than regular Ran‐
dom Forests?**

Meintras que un Random Forest buscar el mejor umbral posible apra cada característica al dividir un nodo, los Extra-Trees eligen umbrales al azar para cada característica y seleccionann el mejor de esos umbrales aleatorios para realizar la división. Este método supondrá un aumentode las vias pero una reducción de la varianza, asi como un aumento en la velocidad de entrenamiento.

**6. If your AdaBoost ensemble underfits the training data, what hyperparameters
should you tweak and how?**

La aparición del underfitting implica que el modelo es demasiado simple para capturar la estructura de los datos por lo que habrá que aumentar la complejidas del modelo a través de: 

- Aumento del n_estimators → incrementar el número de predictores secuenciales para que el modelo tenga más oportunidades de corregir los errores acumulados.
- Reducir la regularización del estimador base → Por ejemplo si usamos un árbol de decisión, aumentar su complejidas ya sea aumentando su profundidad máxima o reduciendo las restricciones de los leaf nodes.
- Aumentar el learning_rate → Ayuda a que cada para de corrección tenga un impacto más fuerte en el resultado final.

**7. If your Gradient Boosting ensemble overfits the training set, should you increase
or decrease the learning rate?**

El learning_rate se encarga de escalar la contribución de cada árbol que se añade al ensamble. Cuando se reduce este valor, cada árbol tiene un impacto menor en el resultado final y es una de las formas más efectivas de regularizar un modelo de Gradiente Boosting y asi evitar el overfitting. 

**8. Load the MNIST data , and split it into a training set, a
validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for val‐
idation, and 10,000 for testing). Then train various classifiers, such as a Random
Forest classifier, an Extra-Trees classifier, and an SVM. Next, try to combine
them into an ensemble that outperforms them all on the validation set, using a
soft or hard voting classifier. Once you have found one, try it on the test set. How
much better does it perform compared to the individual classifiers?